In [1]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
#from tensorflow.keras import optimizers #., optimizers
import pandas as pd
import numpy as np

import tensorflow as tf


#Importing all the relevant library
%matplotlib inline
import h5py, os
#from functions import transforms as T
#from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
#from functions import transforms as T 
#from functions.subsample import MaskFunc
from PIL import Image

In [2]:
def load_data_path(train_data_path, val_data_path, test_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_val_test = ['train', 'val', 'test']
    data_path = [train_data_path, val_data_path, test_data_path]
      
    for i in range(len(data_path)):

        data_list[train_val_test[i]] = []
        
        which_data_path = data_path[i]
        tr = 0
        te = 0
        alfa = 0
        for fname in sorted(os.listdir(which_data_path + '/images')):
            if fname != ".DS_Store":

            
                subject_data_path = os.path.join(which_data_path + '/images', fname)
                     
                if not os.path.isfile(subject_data_path): continue 
            
          #  im_frame = Image.open(subject_data_path)

            #get information from text file
            # this will return a tuple of root and extension
                split_tup = os.path.splitext(fname)

  
            # extract the file name and extension
                file_name = split_tup[0]
                file_path = os.path.join(which_data_path + '/texts', file_name) + '.txt'
                f = open(os.path.join(which_data_path + '/texts', file_name) + '.txt', 'r')
                line = f.readlines()[1]
            
                fields = line.split(',')
                tr = int(fields[3])
                te = int(fields[4])
                alfa = int(fields[5])
                f.close()
                
            # the first 5 slices are mostly noise so it is better to exlude them
                data_list[train_val_test[i]] += [(fname, te)]
    
    return data_list

In [3]:
data_list = load_data_path ("data/train", "data/val", "data/test")

In [4]:
train_data = data_list['train']
val_data = data_list['val']
test_data = data_list['val']

In [5]:
train_df = pd.DataFrame(train_data, columns=['fnames', 'labels'])
train_df['labels']= train_df['labels'].astype(str)
train_df['fnames']= train_df['fnames'].astype(str)
val_df = pd.DataFrame(val_data, columns=['fnames', 'labels'])
val_df['labels']= val_df['labels'].astype(str)
val_df['fnames']= val_df['fnames'].astype(str)
test_df = pd.DataFrame(test_data, columns=['fnames', 'labels'])
test_df['labels']= test_df['labels'].astype(str)
test_df['fnames']= test_df['fnames'].astype(str)
labels = train_df.labels.unique()

In [6]:
num_class = len(labels)
num_class

18

In [7]:
train_df.describe

<bound method NDFrame.describe of         fnames labels
0        1.png     11
1       10.png      3
2      100.png     11
3     1000.png      7
4     1001.png     13
...        ...    ...
3060   995.png      2
3061   996.png      3
3062   997.png     10
3063   998.png      3
3064   999.png      3

[3065 rows x 2 columns]>

In [8]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
      dataframe=train_df,
      directory="./data/train/images/",
      x_col="fnames",
      y_col="labels",
      batch_size=64,
      seed=42,
      shuffle=True,
      class_mode="categorical",
      target_size=(224,224))



valid_generator=test_datagen.flow_from_dataframe(
      dataframe=val_df,
      directory="./data/val/images/",
      x_col="fnames",
      y_col="labels",
      batch_size=64,
      seed=42,
      shuffle=True,
      class_mode="categorical",
     target_size=(224,224))


Found 3065 validated image filenames belonging to 18 classes.
Found 655 validated image filenames belonging to 18 classes.


In [9]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
      directory="./data/test/images",
      x_col="fnames",
      batch_size=1,
      seed=42,
      shuffle=False,
      class_mode=None,
      target_size=(224,224))

Found 646 validated image filenames.


/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 9 invalid image filename(s) in x_col="fnames". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_class, activation='sigmoid'))
model.compile(tf.keras.optimizers.RMSprop(lr=0.001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

2021-11-30 16:59:38.325782: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
n_epochs = 300 #100 #10

In [12]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=n_epochs
)

/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-11-30 16:59:39.574789: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/300
47/47 [==============================] - 252s 5s/step - loss: 0.3885 - accuracy: 0.1387 - val_loss: 0.2063 - val_accuracy: 0.2734
Epoch 2/300
47/47 [==============================] - 240s 5s/step - loss: 0.1985 - accuracy: 0.1966 - val_loss: 0.2016 - val_accuracy: 0.2641
Epoch 3/300
47/47 [==============================] - 239s 5s/step - loss: 0.1807 - accuracy: 0.2542 - val_loss: 0.1612 - val_accuracy: 0.2953
Epoch 4/300
47/47 [==============================] - 240s 5s/step - loss: 0.1613 - accuracy: 0.2959 - val_loss: 0.1507 - val_accuracy: 0.3078
Epoch 5/300
47/47 [==============================] - 243s 5s/step - loss: 0.1469 - accuracy: 0.3478 - val_loss: 0.1441 - val_accuracy: 0.3297
Epoch 6/300
47/47 [==============================] - 242s 5s/step - loss: 0.1403 - accuracy: 0.3707 - val_loss: 0.1386 - val_accuracy: 0.3500
Epoch 7/300
47/47 [==============================] - 243s 5s/step - loss: 0.1336 - accuracy: 0.3756 - val_loss: 0.1255 - val_accuracy: 0.3953
Epoch 

In [13]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


646/646 [==============================] - 38s 59ms/step


In [14]:
pred

array([[5.8603887e-06, 7.1591976e-06, 2.2128458e-07, ..., 1.3970106e-07,
        1.7018463e-07, 2.6283331e-07],
       [8.0961401e-25, 1.3346599e-24, 9.9745268e-01, ..., 1.1962352e-13,
        5.5591663e-08, 2.3077607e-02],
       [2.7306040e-14, 1.0710371e-13, 1.8770956e-09, ..., 1.1225283e-02,
        4.0474743e-02, 8.2398246e-06],
       ...,
       [1.0000000e+00, 3.6453353e-17, 1.9277035e-24, ..., 1.3829804e-24,
        9.1412462e-27, 2.0635383e-23],
       [5.8603887e-06, 7.1591976e-06, 2.2128458e-07, ..., 1.3970106e-07,
        1.7018463e-07, 2.6283331e-07],
       [2.1589161e-25, 7.6994143e-17, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [15]:
pred_bool = (pred >0.5)

In [16]:
predictions=[]
labels = train_generator.class_indices

textfile = open("TE_labels.txt", "w")
for element in labels:
    textfile.write(element + "\n")
textfile.close()
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("TEresults.csv",index=False)

In [17]:
results

,Filename,Predictions
0,1.png,
1,10.png,10
2,100.png,6
3,101.png,2
4,102.png,0
...,...,...
641,95.png,1
642,96.png,2
643,97.png,0
644,98.png,


In [18]:
import PIL
original = PIL.Image.open("data/test/images/100.png")
file_type = original.format

original.save("testing/test.png", format="png")


In [19]:
import PIL
original = PIL.Image.open("data/test/images/100.png") # replace later with image input
file_type = original.format

original.save("testing/test.png", format="png")
##########################
testdata = []
for fname in sorted(os.listdir('testing')):
    if fname == ".DS_Store": continue
            
    subject_data_path = os.path.join('testing', fname)                   
    if not os.path.isfile(subject_data_path): continue          
    testdata.append(fname)
    
df = pd.DataFrame(testdata, columns=['fnames'])
df['fnames']= df['fnames'].astype(str)
test_datagen = ImageDataGenerator(rescale=1./255.)
test_g=test_datagen.flow_from_dataframe(
     dataframe= df,
      directory="./testing",
      x_col="fnames",
      batch_size=1,
      seed=42,
      shuffle=False,
      class_mode=None,
      target_size=(224,224))
STEP_SIZE_TEST=test_g.n//test_g.batch_size
pred = model.predict_generator(test_g,
                               steps=STEP_SIZE_TEST,
                               verbose=1)
pred_bool = (pred >0.5)
predictions=[]
#labels = train_generator.class_indices
#labels = dict((v,k) for k,v in labels.items())
labels = {}
file1 = open('TE_labels.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    labels[count] = line.strip()
    count += 1
    
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
    
    
if predictions[0] == '':
    result = 0
else:
    result = float (predictions[0] )
#result = float (predictions[0] ) 

Found 1 validated image filenames.
1/1 [==============================] - 0s 94ms/step


/Users/hoangvo/opt/anaconda3/envs/coursework/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [20]:
predictions[0]

'6'

In [21]:
result

6.0

In [22]:
labels

{0: '0',
 1: '1',
 2: '10',
 3: '11',
 4: '12',
 5: '13',
 6: '14',
 7: '15',
 8: '16',
 9: '17',
 10: '2',
 11: '3',
 12: '4',
 13: '5',
 14: '6',
 15: '7',
 16: '8',
 17: '9'}

In [23]:
model

In [24]:
# save the model
filename2 = 'model_te.h5' 
model.save(filename2)

In [25]:
from keras.models import load_model
# load model
model = load_model('model_te.h5')

In [66]:
import matlab.engine


ModuleNotFoundError: No module named 'matlab.engine'